# Testing

In [43]:
import pandas as pd
import joblib

Load the model, scaler, and feature list

In [44]:
model = joblib.load("../model/model_v0.pkl")
scaler = joblib.load("../model/scaler_model_v0.pkl")
train_features = joblib.load("../model/train_features_model_v0.pkl")

Load the new dataset + Align columns with the training features

In [45]:
new_df = pd.read_csv("../data/test_without_gt.csv")

ids = new_df["ID"]

drop_cols = ["ID", "Shop Name", "pms_i_ymd", "date_of_birth", "c_postal_code", "postal_code"]
new_df = new_df.drop(columns=drop_cols, errors="ignore")

new_df = pd.get_dummies(new_df, drop_first=True)

In [46]:
for col in train_features:
    if col not in new_df.columns:
        new_df[col] = 0

new_df = new_df[train_features]

Scale

In [47]:
new_scaled = scaler.transform(new_df)

Predict

In [48]:
y_proba = model.predict_proba(new_scaled)[:, 1]

# Use a lower threshold to catch more defaults
threshold = 0.3
preds = (y_proba > threshold).astype(int)

Export output

In [49]:
output = pd.DataFrame({
    "ID": ids,
    "default_12month": preds
})
output.to_csv("../prediction/predictions.csv", index=False)